# KDM SDK 퀵스타트 가이드

이 노트북은 KDM SDK를 사용하여 수자원 데이터를 조회하고 분석하는 방법을 5분 안에 배울 수 있도록 구성되었습니다.

## 목차
1. 설치 및 환경 설정
2. 첫 번째 쿼리 - 댐 저수율 조회
3. 데이터프레임으로 변환
4. 시각화
5. 데이터 내보내기 (Excel, CSV)
6. 배치 쿼리 - 여러 댐 비교
7. 템플릿 사용하기

## 1. 설치 및 환경 설정

터미널에서 다음 명령어로 KDM SDK를 설치하세요:

```bash
pip install kdm-sdk[analyst]
```

분석가용 추가 패키지를 포함합니다 (matplotlib, pandas, openpyxl 등)

In [ ]:
# 필요한 라이브러리 import
import os
from kdm_sdk import KDMQuery
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("✅ KDM SDK 준비 완료!")

## 2. 첫 번째 쿼리 - 소양강댐 저수율

소양강댐의 최근 7일 저수율 데이터를 조회해봅시다.

In [ ]:
# KDM 쿼리 생성
query = KDMQuery()

# 소양강댐 저수율 조회 (최근 7일)
result = await query.dam('소양강댐').measurement('저수율').days(7).get()

# 결과 확인
print(f"조회 성공: {result.success}")
print(f"시설명: {result.site_name}")
print(f"데이터 개수: {len(result)} 개")
print(f"\n첫 3개 레코드:")
for record in result.data[:3]:
    print(f"  {record['datetime']}: {record['values']}")

## 3. 데이터프레임으로 변환

pandas DataFrame으로 변환하면 데이터 분석이 훨씬 쉬워집니다.

In [ ]:
# DataFrame 변환
df = result.to_dataframe()

# 데이터 미리보기
print("📊 데이터프레임 정보:")
print(df.info())
print("\n처음 5개 행:")
df.head()

In [ ]:
# 기본 통계
print("📈 기본 통계:")
df.describe()

## 4. 시각화

matplotlib를 사용하여 데이터를 시각화합니다.

In [ ]:
# 시계열 차트
plt.figure(figsize=(12, 6))
plt.plot(df['datetime'], df['저수율'], marker='o', linewidth=2, markersize=6)
plt.title('소양강댐 저수율 추이 (최근 7일)', fontsize=16, fontweight='bold')
plt.xlabel('날짜', fontsize=12)
plt.ylabel('저수율 (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"✅ 평균 저수율: {df['저수율'].mean():.2f}%")
print(f"✅ 최고: {df['저수율'].max():.2f}%, 최저: {df['저수율'].min():.2f}%")

## 5. 데이터 내보내기

Excel, CSV 등 다양한 형식으로 데이터를 내보낼 수 있습니다.

In [ ]:
# Excel로 내보내기
result.to_excel('soyang_storage.xlsx', sheet_name='소양강댐')
print("✅ Excel 파일 생성: soyang_storage.xlsx")

# CSV로 내보내기 (한글 인코딩 자동 처리)
result.to_csv('soyang_storage.csv')
print("✅ CSV 파일 생성: soyang_storage.csv")

# JSON으로 내보내기
result.to_json('soyang_storage.json', indent=2)
print("✅ JSON 파일 생성: soyang_storage.json")

## 6. 배치 쿼리 - 여러 댐 비교

여러 댐의 데이터를 한 번에 조회하여 비교할 수 있습니다.

In [ ]:
# 여러 댐의 저수율 조회
batch_query = KDMQuery()
batch_query.facility_type('dam')
batch_query.measurement('저수율')
batch_query.days(7)

# 3개 댐 추가
batch_query.add_site('소양강댐')
batch_query.add_site('충주댐')
batch_query.add_site('대청댐')

# 배치 실행
batch_results = await batch_query.execute_batch()

print(f"✅ 조회 완료: {len(batch_results)}개 댐")
for site_name, result in batch_results:
    if result.success:
        print(f"  - {site_name}: {len(result)}개 데이터")

In [ ]:
# 배치 결과를 하나의 DataFrame으로 통합
combined_df = batch_results.aggregate()

print("📊 통합 데이터프레임:")
print(combined_df.head(10))
print(f"\n총 {len(combined_df)}개 레코드")

In [ ]:
# 댐별 평균 저수율 비교
avg_by_dam = combined_df.groupby('site_name')['저수율'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
avg_by_dam.plot(kind='bar', color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.title('댐별 평균 저수율 비교', fontsize=16, fontweight='bold')
plt.xlabel('댐명', fontsize=12)
plt.ylabel('평균 저수율 (%)', fontsize=12)
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n📊 댐별 평균 저수율:")
for dam, avg in avg_by_dam.items():
    print(f"  {dam}: {avg:.2f}%")

In [ ]:
# 배치 결과를 Excel로 내보내기
batch_results.to_excel('dam_comparison.xlsx', sheet_name='댐 비교')
print("✅ Excel 파일 생성: dam_comparison.xlsx")

## 7. 템플릿 사용하기

자주 사용하는 쿼리 패턴을 템플릿으로 저장하고 재사용할 수 있습니다.

In [ ]:
from kdm_sdk.templates import TemplateBuilder

# 템플릿 생성
template = (
    TemplateBuilder()
    .name("주간 댐 저수율 리포트")
    .description("주요 댐의 주간 저수율 모니터링")
    .facility_type("dam")
    .measurement(["저수율", "유입량", "방류량"])
    .days(7)
    .sites(["소양강댐", "충주댐", "대청댐"])
    .build()
)

print("✅ 템플릿 생성 완료")
print(f"템플릿 이름: {template.name}")
print(f"설명: {template.description}")

In [ ]:
# 템플릿으로 쿼리 실행
query_from_template = template.to_query()
template_results = await query_from_template.execute_batch()

print(f"✅ 템플릿 실행 완료: {len(template_results)}개 댐")

# 결과 확인
for site_name, result in template_results:
    if result.success:
        df = result.to_dataframe()
        print(f"\n{site_name}:")
        print(f"  데이터 개수: {len(df)}")
        print(f"  컬럼: {list(df.columns)}")

In [ ]:
# 템플릿을 YAML 파일로 저장
template.save_yaml('weekly_dam_report.yaml')
print("✅ 템플릿 저장: weekly_dam_report.yaml")
print("\n나중에 이 템플릿을 다시 불러올 수 있습니다:")
print("from kdm_sdk.templates import Template")
print("template = Template.load_yaml('weekly_dam_report.yaml')")

## 🎉 완료!

축하합니다! KDM SDK의 기본 사용법을 익혔습니다.

### 다음 단계

1. **더 많은 예제**: `examples/` 디렉토리의 다른 예제들을 확인하세요
2. **API 문서**: `docs/API_OVERVIEW.md`에서 전체 API를 확인하세요
3. **레시피 북**: `docs/RECIPE_BOOK.md`에서 50+ 복사-붙여넣기 가능한 레시피를 확인하세요
4. **고급 기능**:
   - 상관관계 분석: `correlation_analysis.ipynb`
   - 시계열 분석: `time_series_recipes.ipynb`
   - 배치 분석: `batch_analysis.ipynb`

### 도움이 필요하신가요?

- GitHub Issues: [kdm-sdk/issues](https://github.com/kwater/kdm-sdk/issues)
- 문서: `docs/ANALYST_QUICKSTART.md`

Happy analyzing! 🚀